In [ ]:
%%shell
pip3 install transformers>=4.32.0 optimum>=1.12.0
pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 24.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model and tokenizer as before

def preprocess_text(multi_para_text):
    # Replace newline characters with spaces to create a single line of text
    return multi_para_text.replace('\n', ' ').strip()

def generate_summary(text, max_words):
    # Preprocess the input text to handle multiple paragraphs
    processed_text = preprocess_text(text)

    prompt = f'''Summarize the following text in about {max_words} words:
{processed_text}'''

    prompt_template = f'''You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=4000,  # Adjust based on model capability
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

    response = pipe(prompt_template)[0]['generated_text']
    generated_summary = response[len(prompt_template):].strip()

    # Post-process to limit to max_words, attempting to preserve meaning
    words = generated_summary.split()
    if len(words) > max_words:
        truncated_summary = ' '.join(words[:max_words])
        last_period = truncated_summary.rfind('.')
        if last_period != -1 and last_period + 1 < len(truncated_summary):
            truncated_summary = truncated_summary[:last_period + 1]
    else:
        truncated_summary = generated_summary

    return truncated_summary

# Example usage
text_to_summarize = '''Cricket, often regarded as more than a sport, is a cultural phenomenon that transcends boundaries and unites nations in a shared passion. Originating in England in the 16th century, cricket has evolved into a global spectacle with a fanatical following. Its unique blend of strategy, skill, and teamwork has turned it into one of the most popular sports, captivating millions around the world. The game is played between two teams, each consisting of eleven players, and unfolds on an oval-shaped field with a 22-yard-long pitch at its center.

Central to cricket is the duel between bat and ball, epitomized by the bowler and batsman. The bowler, armed with the objective to dismiss the batsman, hurls the ball towards the batsman, who, armed with a wooden bat, endeavors to strike the ball away. This contest is a microcosm of the broader narrative that unfolds during a cricket match, where strategy, technique, and mental fortitude come to the fore.

International cricket, governed by the International Cricket Council (ICC), showcases the pinnacle of the sport. Tournaments like the ICC Cricket World Cup, held every four years, and various bilateral series captivate global audiences, turning players into heroes and moments into legends. Cricket's unique formats add to its allure, with Test matches, lasting up to five days, emphasizing endurance and technique, One Day Internationals (ODIs) providing a balance of strategy and urgency, and the fast-paced T20 (Twenty20) format catering to the demands of the contemporary, time-conscious audience.

Cricket's influence extends beyond the boundary ropes, shaping cultures and identities. In countries like India, Pakistan, Australia, England, and the West Indies, cricket is more than a game—it's a way of life. The roar of the crowd in iconic stadiums like Lord's in London, the MCG in Melbourne, or Eden Gardens in Kolkata resonates with history and passion. Cricketing rivalries, such as the Ashes between England and Australia or the fierce contests between India and Pakistan, are etched into the collective memory of cricket enthusiasts, becoming cultural touchstones that transcend the sport itself.

The sport's narrative is also woven with individual brilliance and iconic moments. From Sir Don Bradman's unparalleled batting records to Shane Warne's mesmerizing spin wizardry, from Sachin Tendulkar's mastery with the bat to Brian Lara's record-breaking innings, cricket has witnessed legends who have left an indelible mark on the game. The sport's charm lies in its ability to produce heroes in different roles—the stylish batsman, the cunning bowler, the agile wicketkeeper, and the inspirational captain.

Beyond the glamour of international cricket, the game flourishes at the grassroots level, providing a platform for aspiring cricketers to hone their skills and dream of donning their national colors. Cricket academies, local leagues, and school competitions nurture young talent, contributing to the sport's enduring legacy.

In recent years, the advent of Twenty20 leagues, such as the Indian Premier League (IPL), has brought a new dimension to the game, blending entertainment with high-quality cricket. These leagues attract top players from around the world, creating a melting pot of talent and diverse playing styles. The league format, characterized by electrifying performances and nail-biting finishes, has broadened cricket's appeal, attracting a wider and more diverse audience.

In conclusion, cricket is not merely a game; it's a cultural phenomenon that weaves together tradition and modernity, strategy and flair, and national pride and global camaraderie. As the cricketing world continues to evolve, the spirit of the game remains rooted in the joy of competition, the thrill of a well-fought contest, and the unifying power of a sport that knows no boundaries.'''
max_words = 360  # Desired word count
summary = generate_summary(text_to_summarize, max_words)
print('Summary:\n' + summary)




Summary:
Cricket is a cultural phenomenon that transcends boundaries and unites nations with a shared passion. It originated in England in the 16th century and has evolved into a global spectacle with a fanatical following. Cricket is played between two teams of eleven players on an oval-shaped field with a central 22-yard-long pitch. The game revolves around the duel between the bowler and batsman, with strategy, skill, and teamwork coming to the fore. International cricket, governed by the International Cricket Council (ICC), showcases the pinnacle of the sport through tournaments like the ICC Cricket World Cup and various bilateral series. Cricket's unique formats, including Test matches, One Day Internationals (ODIs), and T20 (Twenty20) format, cater to different audiences. Cricket's influence extends beyond the boundary ropes, shaping cultures and identities, particularly in countries like India, Pakistan, Australia, England, and the West Indies. The sport's narrative is woven wit

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

# Reference summary (ground truth)
reference_summary = '''Cricket, originating in 16th century England, has evolved into a global spectacle, captivating millions with its strategic complexity and passionate rivalries. The game, played between two teams of eleven players, symbolizes a captivating duel between bat and ball, where bowlers strive to dismiss batsmen in a microcosm of the broader narrative unfolding during a match. Governed by the International Cricket Council (ICC), the pinnacle of the sport is showcased in tournaments like the ICC Cricket World Cup and various bilateral series, with diverse formats ranging from the enduring Test matches to the urgency of One Day Internationals (ODIs) and the fast-paced Twenty20 (T20) format.Cricket's influence extends far beyond the playing field, shaping cultures and identities in nations like India, Pakistan, Australia, England, and the West Indies. The sport becomes a way of life, with iconic stadiums like Lord's, the MCG, and Eden Gardens echoing with history and passion. Individual brilliance and iconic moments, from Sir Don Bradman's batting records to Shane Warne's spin wizardry and Sachin Tendulkar's mastery, contribute to cricket's enduring legacy.The narrative of cricket also includes grassroots development, with academies, local leagues, and school competitions nurturing young talent. In recent years, the advent of Twenty20 leagues, notably the Indian Premier League (IPL), has revolutionized the game, attracting a diverse audience with its blend of entertainment and high-quality cricket.In conclusion, cricket is more than a game; it's a cultural phenomenon weaving tradition and modernity, strategy and flair, and national pride and global camaraderie. The sport's enduring charm lies in producing heroes across various roles and inspiring a sense of unity. As cricket continues to evolve, its spirit remains rooted in the joy of competition, the thrill of a well-fought contest, and the unifying power that transcends borders, making cricket a testament to the enduring appeal of sport on a global scale.'''

# Generated summary
generated_summary = summary

# Calculate ROUGE scores
rouge = Rouge()
scores = rouge.get_scores(generated_summary, reference_summary)

# Print ROUGE-1, ROUGE-2, and ROUGE-L scores
print("ROUGE-1: ", scores[0]['rouge-1'])
print("ROUGE-2: ", scores[0]['rouge-2'])
print("ROUGE-L: ", scores[0]['rouge-l'])


ROUGE-1:  {'r': 0.40298507462686567, 'p': 0.5472972972972973, 'f': 0.4641833762041363}
ROUGE-2:  {'r': 0.2053872053872054, 'p': 0.30198019801980197, 'f': 0.24448897313713608}
ROUGE-L:  {'r': 0.39800995024875624, 'p': 0.5405405405405406, 'f': 0.45845271717834835}


In [ ]:
# ROUGE scores
rouge_1_f1 = scores[0]['rouge-1']['f']
rouge_2_f1 = scores[0]['rouge-2']['f']
rouge_l_f1 = scores[0]['rouge-l']['f']

# Print ROUGE scores in the specified format
print("ROUGE-1: {:.2f}".format(rouge_1_f1))
print("ROUGE-2: {:.2f}".format(rouge_2_f1))
print("ROUGE-L: {:.2f}".format(rouge_l_f1))

ROUGE-1: 0.46
ROUGE-2: 0.24
ROUGE-L: 0.46


In [ ]:
from transformers import pipeline

def generate_key_points(summary, model, tokenizer, target_length):
    # Define the task prompt
    prompt = f"Create bullet points summarizing the following text:\n{summary}"

    # Set up the pipeline with the model and tokenizer
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=target_length,  # Control the length based on target_length
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

    # Generate the response
    response = pipe(prompt)[0]['generated_text']

    # Post-process to extract the bullet points
    key_points = response[len(prompt):].strip()

    return key_points

# Using the function with the given summary and the loaded model
generated_key_points = generate_key_points(summary, model, tokenizer, len(summary.split()))
print('Key Points:\n' + generated_key_points)


Key Points:
Bullet Points:

* Cricket is a cultural phenomenon with a global following.
* It originated in England in the 16th century and has evolved into a global sport.
* The game is played between two teams of eleven players on an oval-shaped field.
* The duel between the bowler and batsman is the central aspect of the game.
* International cricket is governed by the International Cricket Council (ICC).
* Cricket has unique formats like Test matches, ODIs, and T20.
* Cricket's influence extends beyond the boundary ropes, shaping cultures and identities.
* The sport produces individual brilliance and iconic moments.
* Cricket flourishes at the grassroots level through local leagues and academies.
* Twenty20 leagues like the Indian Premier League (IPL) bring a new dimension to the game.
